In [72]:
import numpy as np
from matplotlib import pyplot as plt
import math

img = np.arange(0,255,4).reshape((8, 8))
print(img)

[[  0   4   8  12  16  20  24  28]
 [ 32  36  40  44  48  52  56  60]
 [ 64  68  72  76  80  84  88  92]
 [ 96 100 104 108 112 116 120 124]
 [128 132 136 140 144 148 152 156]
 [160 164 168 172 176 180 184 188]
 [192 196 200 204 208 212 216 220]
 [224 228 232 236 240 244 248 252]]


In [73]:
# covariance matrix irrelevant for now
np.cov(img, rowvar = False, bias = True)

array([[5376., 5376., 5376., 5376., 5376., 5376., 5376., 5376.],
       [5376., 5376., 5376., 5376., 5376., 5376., 5376., 5376.],
       [5376., 5376., 5376., 5376., 5376., 5376., 5376., 5376.],
       [5376., 5376., 5376., 5376., 5376., 5376., 5376., 5376.],
       [5376., 5376., 5376., 5376., 5376., 5376., 5376., 5376.],
       [5376., 5376., 5376., 5376., 5376., 5376., 5376., 5376.],
       [5376., 5376., 5376., 5376., 5376., 5376., 5376., 5376.],
       [5376., 5376., 5376., 5376., 5376., 5376., 5376., 5376.]])

In [74]:
def correlation(img):
    return np.corrcoef(img, rowvar = False)

In [75]:
def homogeneity(img):
    res = 0
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            res += img[i,j] / (1 + abs(j - i))
    print(res)

In [76]:
def entropy(img):
    res = 0
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            res += (img[i,j]+0.01) * math.log(img[i,j]+0.01) # If pixel value is 0 log is infinite
            #  add
    return res * -1

In [77]:
def variance(img):
    return np.var(img)

In [78]:
def contrast(img):
    res = 0
    res1 = 0
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            n = math.pow(i-j,2)
            res += img[j,i] * n
    return res

In [79]:
def longRunEmphasis(img):
    return 1

In [80]:
def runePercentage(img):
    return 1

In [81]:
def var_mtx(img):
    return np.var(img,axis=0), np.var(img,axis=1)

In [82]:
import pandas as pd

X = pd.DataFrame(columns=[
    'target',
    'correlation',
    'homogeneity',
    'entropy',
    'variance',
    'contrast',
    'longRunEmphasis',
    'runePercentage'
])

target = 'normal'
corr = correlation(img)
homo = homogeneity(img)
ent = entropy(img)
var = variance(img)
cont = contrast(img)
lre = longRunEmphasis(img)
rp = runePercentage(img)

row = {
    'target': target,
    'correlation': corr,
    'homogeneity': homo,
    'entropy': ent,
    'variance': var,
    'contrast': cont,
    'longRunEmphasis': lre,
    'runePercentage': rp
    }
data.append(row,ignore_index=True)

3140.1


,correlation,homogeneity,entropy,variance,contrast,longRunEmphasis,runePercentage,target
0,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1....",None,-40626.159692,5460.0,84672.0,1,1,normal
